In [1]:
import tensorflow as tf
from PIL import Image
tf.__version__

'2.0.0'

In [21]:
### from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os

In [3]:
from mnist_decode import decode_idx3_ubyte,decode_idx1_ubyte

x_train = decode_idx3_ubyte('./input/coic/train-images-idx3-ubyte')
y_train = decode_idx1_ubyte('./input/coic/train-labels-idx1-ubyte')
print(x_train.shape,y_train.shape)
x_test = decode_idx3_ubyte('./input/coic/test-images-idx3-ubyte')
y_test = decode_idx1_ubyte('./input/coic/test-labels-idx1-ubyte')
print(x_test.shape,y_test.shape)

(18225, 28, 28) (18225,)
(4563, 28, 28) (4563,)


In [4]:
from keras.utils import np_utils
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32')
x_train = x_train/255
x_test = x_test/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(x_test.shape,y_test.shape)

(4563, 1, 28, 28) (4563, 10)


Using TensorFlow backend.


In [5]:

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds


<TensorSliceDataset shapes: ((1, 28, 28), (10,)), types: (tf.float32, tf.float32)>

In [6]:

train_ds = train_ds.shuffle(10000).batch(32)
# suffle表示隨機選取,並分為32batch
train_ds


<BatchDataset shapes: ((None, 1, 28, 28), (None, 10)), types: (tf.float32, tf.float32)>

In [7]:

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.shuffle(10000).batch(32)


In [8]:
num_classes = y_test.shape[1]
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu',data_format='channels_first'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [10]:

# Fit the model
model.fit(x_test, y_test, validation_data=(x_test, y_test), epochs=30, batch_size=200)


Train on 4563 samples, validate on 4563 samples
Epoch 1/30
4563/4563 [==============================] - 0s 59us/sample - loss: 0.0313 - accuracy: 0.9888 - val_loss: 0.0116 - val_accuracy: 0.9958
Epoch 2/30
4563/4563 [==============================] - 0s 56us/sample - loss: 0.0173 - accuracy: 0.9939 - val_loss: 0.0075 - val_accuracy: 0.9980
Epoch 3/30
4563/4563 [==============================] - 0s 57us/sample - loss: 0.0075 - accuracy: 0.9978 - val_loss: 0.0037 - val_accuracy: 0.9998
Epoch 4/30
4563/4563 [==============================] - 0s 57us/sample - loss: 0.0059 - accuracy: 0.9991 - val_loss: 0.0032 - val_accuracy: 0.9998
Epoch 5/30
4563/4563 [==============================] - 0s 56us/sample - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 6/30
4563/4563 [==============================] - 0s 58us/sample - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 7/30
4563/4563 [==============================] - 0s 56us/sample

In [26]:
x_final_test = decode_idx3_ubyte('./input/coic/final-test-images-idx3-ubyte')
x_final_test = x_final_test.reshape(x_final_test.shape[0], 1, 28, 28).astype('float32')
x_final_test = x_final_test/255
ans=model.predict(x_final_test)

In [35]:
tmp = []
l=[]
for i,pre in enumerate(ans):
    l = [i+1,pre.argmax()]
    tmp.append(l)

In [36]:
print(pd.DataFrame(tmp))

         0  1
0        1  2
1        2  7
2        3  2
3        4  2
4        5  1
5        6  3
6        7  6
7        8  4
8        9  7
9       10  1
10      11  3
11      12  4
12      13  6
13      14  3
14      15  2
15      16  9
16      17  7
17      18  3
18      19  9
19      20  4
20      21  9
21      22  3
22      23  2
23      24  0
24      25  3
25      26  9
26      27  7
27      28  2
28      29  9
29      30  0
...    ... ..
4533  4534  7
4534  4535  7
4535  4536  1
4536  4537  3
4537  4538  7
4538  4539  0
4539  4540  4
4540  4541  4
4541  4542  8
4542  4543  3
4543  4544  2
4544  4545  3
4545  4546  7
4546  4547  9
4547  4548  0
4548  4549  8
4549  4550  3
4550  4551  1
4551  4552  0
4552  4553  9
4553  4554  6
4554  4555  3
4555  4556  2
4556  4557  7
4557  4558  1
4558  4559  0
4559  4560  2
4560  4561  8
4561  4562  0
4562  4563  0

[4563 rows x 2 columns]


In [37]:
sub =  pd.DataFrame(tmp)
sub = sub.rename(index=str, columns={0: "id", 1: "Class"})
sub.to_csv('submission1.csv', index=False)